In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from haversine import haversine, Unit

In [2]:
df = pd.read_csv('mergeddate.csv')

In [3]:
df.drop(columns = ['Unnamed: 0', 'is_closed', 'price'], inplace = True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365 entries, 0 to 1364
Data columns (total 21 columns):
alias                       1365 non-null object
categories                  1365 non-null object
coordinates                 1365 non-null object
display_phone               1303 non-null object
location                    1365 non-null object
name                        1365 non-null object
rating                      1365 non-null float64
review_count                1365 non-null int64
transactions                1365 non-null object
url                         1365 non-null object
permanently closed          1365 non-null int64
outdoor_seating             1365 non-null int64
price_cat                   1365 non-null object
delivery                    1365 non-null int64
restaurant_reservation      1365 non-null int64
pickup                      1365 non-null int64
zip_code                    1365 non-null int64
latitude                    1365 non-null float64
longitude       

In [45]:
df.corrwith(df['permanently closed'])

rating                     -0.234103
review_count               -0.024119
permanently closed          1.000000
outdoor_seating            -0.362192
delivery                   -0.182721
restaurant_reservation     -0.083764
pickup                     -0.168543
zip_code                   -0.105011
latitude                   -0.024536
longitude                  -0.119590
total_restaurants_in_zip    0.217481
Population Density          0.066126
distance_to_subway         -0.027190
dtype: float64

In [155]:
df['Population Density ']/df.total_restaurants_in_zip

0         99.521197
1        277.063087
2        451.735714
3        712.784252
4        265.225581
           ...     
1360      94.867401
1361      79.147361
1362     216.941935
1363     138.653782
1364    1723.308955
Length: 1365, dtype: float64

In [6]:
subwaydf = pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')

In [7]:
subwaydf = subwaydf[['NAME', 'the_geom']]

In [8]:
subwaydf['longitude'] = subwaydf.the_geom.apply(lambda x: x.split()[1][1:])
subwaydf['latitude'] = subwaydf.the_geom.apply(lambda x: x.split()[2][:-1])

In [9]:
df.longitude = df.longitude.apply(lambda x: float(x))
df.latitude = df.latitude.apply(lambda x: float(x))
subwaydf.longitude = subwaydf.longitude.apply(lambda x: float(x))
subwaydf.latitude = subwaydf.latitude.apply(lambda x: float(x))

In [32]:
def find_nearest(restaurant, lon, lat, list_of_coordinates): 
    p = {}
    for i in list_of_coordinates:
        p.update({i: haversine((lon, lat), i)})
    for j in p:
        if p[j] == min(p.values()):
            return {restaurant : min(p.values())}

In [39]:
def nearest_columns(restaurants, longs, lats, coordinates):
    q = {}
    for k in range(0, len(longs)):
        q.update(find_nearest(restaurants[k], longs[k], lats[k], coordinates))
    return q

In [41]:
q = nearest_columns(df.alias, df.longitude, df.latitude, subwaycoords)

In [42]:
df['distance_to_subway'] = df.alias.apply(lambda x: q.get(x))

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365 entries, 0 to 1364
Data columns (total 23 columns):
alias                       1365 non-null object
categories                  1365 non-null object
coordinates                 1365 non-null object
display_phone               1303 non-null object
location                    1365 non-null object
name                        1365 non-null object
rating                      1365 non-null float64
review_count                1365 non-null int64
transactions                1365 non-null object
url                         1365 non-null object
permanently closed          1365 non-null int64
outdoor_seating             1365 non-null int64
price_cat                   1365 non-null object
delivery                    1365 non-null int64
restaurant_reservation      1365 non-null int64
pickup                      1365 non-null int64
zip_code                    1365 non-null int64
latitude                    1365 non-null float64
longitude       

In [28]:
subwaycoords = list(zip(subwaydf.longitude, subwaydf.latitude))

In [197]:
g = nearest_columns(df.longitude[:14], df.latitude[:14], subwaycoords)

-73.982387

In [136]:
q = {}
for k in range(0, len(df.longitude)):
        q.update(find_nearest(df.longitude[k], df.latitude[k], list(zip(subwaydf.longitude, subwaydf.latitude))))

In [100]:
q = {}
for i in range(0, len(df.longitude)):
    pt = (df.longitude[i], df.latitude[i])
    p = {}
    for j in range(0, len(subwaydf.longitude)):
        p.update({subwaydf.NAME[j] : haversine(pt, (subwaydf.longitude[j], subwaydf.latitude[j]))})
        for k in p.keys():
            if p[k] == min(p.values()):
                q.update({k : min(p.values())})
            

In [108]:
min(p.values())

0.3801521244931524

In [110]:
len(subwaydf.longitude)

473